#### Задание 1**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг
Результат классификации запишите в столбец class


In [3]:
import pandas as pd
rate = pd.read_csv('ratings.csv')

class_rate = {2 : 'низкий рейтинг', 
              4 : 'средний рейтинг',
              5 : 'высокий рейтинг'}

def def_rate(row):
    for key, value in class_rate.items():
        if row['rating'] <= key:
            return value
        
rate['class'] = rate.apply(def_rate, axis = 1)
rate.tail(20)

,userId,movieId,rating,timestamp,class
99984,671,5010,2.0,1066793004,низкий рейтинг
99985,671,5218,2.0,1065111990,низкий рейтинг
99986,671,5299,3.0,1065112004,средний рейтинг
99987,671,5349,4.0,1065111863,средний рейтинг
99988,671,5377,4.0,1064245557,средний рейтинг
99989,671,5445,4.5,1064891627,высокий рейтинг
99990,671,5464,3.0,1064891549,средний рейтинг
99991,671,5669,4.0,1063502711,средний рейтинг
99992,671,5816,4.0,1065111963,средний рейтинг
99993,671,5902,3.5,1064245507,средний рейтинг


#### Задание 2
Используем файл keywords.csv.
Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую 
принадлежность определенному региону.   
Т. е. если поисковый запрос содержит название города региона, 
то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.
Результат классификации запишите в отдельный столбец region.

In [4]:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}

import pandas as pd
keywords = pd.read_csv('keywords.csv')

def region_name(row):
    for key, value in geo_data.items():
        for i in value:
            if i in row['keyword']:
                return key
    return 'undefined'

keywords['region'] = keywords.apply(region_name, axis = 1)
keywords[(keywords['region'] != 'undefined')].head(5)

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток


#### Задание 3
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, 
используя файлы с рейтингами фильмов из прошлого домашнего занятия 
(файл movies.csv и ratings.csv из базы grouplens.org.../movielens).  
Т.е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.
При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:
В переменную years запишите список из всех годов с 1950 по 2010.
Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска.
Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’
Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [7]:
years = list(range(1950, 2011))

ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
joined = ratings.merge(movies, on = 'movieId', how = 'left')

import re

def production_year(row):
    for i in joined.title:
        if re.findall(r'\d{4}', row['title']) != []:
            return str(re.findall(r'\d{4}', row['title'])[0])
    return '1900'

joined['year'] = joined.apply(production_year, axis = 1)

result = {}

for k in years:
    dict = {k : joined[joined['year'] == str(k)].rating.mean()}
    result.update(dict)

# создание датафрейма из словаря

new_df = pd.DataFrame.from_dict(result, orient='index', columns = ['mean_rating'])
new_df.sort_values(by = ['mean_rating'], ascending = False)

,mean_rating
1957,4.014241
1972,4.011136
1952,4.000000
1974,3.999058
1954,3.994220
...,...
2005,3.448434
2003,3.445843
1996,3.422675
1997,3.416934
